<img src="https://github.com/LinkedEarth/Logos/blob/master/PyLiPD/pyLiPD_logo1_transparent.png?raw=true" width ="800">

# Creating LiPD files from a tabular template

## Authors

[Deborah Khider](https://orcid.org/0000-0001-7501-8430)


## Preamble

If you are planning to only create one LiPD file on your own, we recommend using the [LiPD Playground](https://lipd.net/playground). This tutorial is intended for users who wish to programatically create multiple files from a template. 

### Goals

* Create a LiPD formatted Dataset from an excel template
* Adding an ensemble table 
* Save the Dataset to a file

Reading Time: 10 minutes

### Keywords

LiPD, LinkedEarth Ontology, Object-Oriented Programming

### Pre-requisites

An understanding of OOP and the LinkedEarth Ontology. Completion of [Dataset class example](L3_dataset_class.ipynb). An understanding how to [edit LiPD files](L3_editing.ipynb) can also be useful. 

## Data Description

- Tian, Y., Fleitmann, D., Zhang, Q., Sha, L., Wassenburg, J. A., Axelsson, J., … Cheng, H. (2023). Holocene climate change in southern Oman deciphered by speleothem records and climate model simulations. Nature Communications, 14(1), 4718. doi:[10.1038/s41467-023-40454-z](https://www.nature.com/articles/s41467-023-40454-z). 

## Demonstration

Let's import the necessary packages. 